## Jupyter Notebook zur Erstellung des Lehrer-Subsets

Die Fragebögen aus den Maurer- und DWA- Befragungen wurden an Lehrkräfte im Zielgebiet verschickt. Gegeben der kurzen Zeitspanne zwischen den Untersuchungen und der Vielzahl der Fragebögen, liegt die Vermutung nahe, dass einige Lehrkräfte an beiden Befragungen teilgenommen haben. Es soll geprüft werden, ob die sich die Variablität zwischen den Befragungen verringert, wenn die selbe Person die Bögen beantwortet hat (Varianzreduktion in Bezug auf persönlich Schreibpräferenz, Angewohnheiten etc.)

Uns liegt eine Tabelle vor, in der für jeden Befragungsort geschaut wurde, ob die Lehrkraft übereinstimmt oder nicht. Dabei ist zu beachten, dass lediglich die transkribierten Lehrernamen aus der DWA Befragung vorliegen. Die Lehrernamen der Maurerbefragung wurden bis jetzt noch nicht transliteriert. Außerdem werden Mehrfachnennungen von Lehrkräften in einem Bogen in der Tabelle auf mehrere Spalten aufgeteilt. Das führt dazu, dass die Fragebögen (sowohl DWA als auch Maurer) oft mehrfach in der Tabelle vorkommen und die Orte entsprechend auch mehrfach vorkommen.

Um diese Informationen sinnvoll nutzen zu können, muss ein Subset des Datensatzes erstellt werden. Dieses Subset soll lediglich die Orte enthalten, an denen eine Übereinstimmung der Lehrkraft vorliegt. Deshalb wird zunächst die Lehrertabelle eingelesen, um Nicht-Übereinstimmungen bereinigt und zu Letzt um die Worte aus dem Fragebögen ergänzt. Auf dieser Basis werden wieder Levenshtein Distanzen berechnet.

Vermutlich gäbe es einen effizienteren Weg, diese Aufgabe zu lösen, als hier geschehen.

In [1]:
# Lehrertabelle einlesen, kürzen und transponieren
import pandas as pd
import numpy as np
from Levenshtein import distance

df_lehrer = pd.read_csv("/home/kopatsch/Masterarbeit/MA/masterarbeit/data/csv_teacher/DWA_Maurer_Lehrerabgleich.csv", sep = "\t")

df_lehrer = df_lehrer.drop(df_lehrer[df_lehrer["Lehrer.übereinstimmend"] != "ja"].index)
df_lehrer = df_lehrer.drop(["lfd_Maurer", "Bearbeiten/in_DWA", "Dubletten.DWA", "überprüft.von", "Name.d..Lehrers_DWA", "Geburtsort.d..Lehrers_DWA", "Korrektur", "Lehrer.übereinstimmend_clean"], axis = 1)

df_words = pd.read_csv("/home/kopatsch/Masterarbeit/MA/masterarbeit/data/csv_gesamt/all_words.csv", sep = "\t")

words = ["ameise", "begräbnis", "deichsel", "elster", "fledermaus", "gurke", "hagebutte", "hebamme", "kartoffel", "maulwurf", "pflaume", "stecknadel", "ziege", "zimmerfliege"]

In [2]:
def leven(val1, val2, ziel):
    """
    computes Levenshtein distance for two given values

    Parameters:
    val1(str or list): list must be comprised of str elements. Function checks if parameter is str or list. 
    If parameter is list, function will compute Levenshtein distance for each str within list.
    val2(str or list): list must be comprised of str elements. 
    Function checks if parameter is str or list. If parameter is list, function will compute Levenshtein distance for each str within list.

    Returns:
    minimum (int): minimal Levenshein distance computed for two given values
    """
    val1 = str(val1)
    val2 = str(val2)
    
    if ", " in val1:
        wert1 = val1.split(", ")
    else:
        wert1 = val1.strip()

    if ", " in val2:
        wert2 = val2.split(", ")
    else:
        wert2 = val2.strip()

    dist = []

    if type(wert1) == str and type(wert2) == str:
        if wert1 == "na" or wert1 == "999" or wert1 == "k.a." or wert1 == "nan" or wert2 == "na" or wert2 == "999" or wert2 == "k.a." or wert2 == "nan":
            dist_item = 999
        else:  
            dist_item = distance(wert1, wert2)
        dist.append(dist_item)

    elif type(wert1) == str and type(wert2) == list:
        for a in wert2:
            a = a.strip()
            if wert1 == "na" or wert1 == "999" or wert1 == "k.a." or wert1 == "nan" or a == "na" or a == "999" or a == "k.a." or a == "nan":
                dist_item = 999
            else:  
                dist_item = distance(wert1, a)
            dist.append(dist_item)

    elif type(wert2) == str and type(wert1) == list:
        for c in wert1:
            c = c.strip()
            if c == "na" or c == "999" or c == "k.a." or c == "nan" or wert2 == "na" or wert2 == "999" or wert2 == "k.a." or wert2 == "nan":
                dist_item = 999
            else:
                dist_item = distance(wert2, c)
            dist.append(dist_item)
    
    elif type(wert1) == list and type(wert2) == list:
        for l in wert1:
            l = l.strip()
            for b in wert2:
                b = b.strip()
                if l == "na" or l == "999" or l == "k.a." or l == "nan" or b == "na" or b == "999" or b == "k.a." or b == "nan":
                    dist_item = 999
                else:
                    dist_item = distance(l, b)
                dist.append(dist_item)
    
    minimum = min(dist)
    ziel.append(minimum)

In [3]:
maurer_bogen = set(df_lehrer["Fragebogen-Nr._Maurer"])
dwa_bogen = set(df_lehrer["Digi_Index"])

maurer = df_words.loc[df_words["Fragebogen Nr."].isin(maurer_bogen)]
maurer = maurer.drop(["Unnamed: 0", "Longitude", "Latitude", "Erhebung"], axis = 1)
maurer = maurer.reset_index(drop=True)
maurer = maurer.rename({"Fragebogen Nr.": "Fragebogen-Nr._Maurer"}, axis = "columns")
for word in words:
    maurer = maurer.rename({f"org {word}": f"maurer_org_{word}",
                            f"item {word}": f"maurer_item_{word}",
                            f"phontype {word}": f"maurer_phontype_{word}",
                            f"lextype {word}": f"maurer_lextype_{word}"}, axis = "columns")
dwa = df_words.loc[df_words["Fragebogen Nr."].isin(dwa_bogen)]
dwa = dwa.drop(["Unnamed: 0", "Longitude", "Latitude", "Erhebung"], axis = 1)
dwa = dwa.reset_index(drop=True)
dwa = dwa.rename({"Fragebogen Nr.": "Digi_Index"}, axis = "columns")
for word in words:
    dwa = dwa.rename({f"org {word}": f"dwa_org_{word}",
                            f"item {word}": f"dwa_item_{word}",
                            f"phontype {word}": f"dwa_phontype_{word}",
                            f"lextype {word}": f"dwa_lextype_{word}"}, axis = "columns")

In [4]:
df_gesamt = pd.merge(df_lehrer, maurer, how = "outer", on = ["Fragebogen-Nr._Maurer"])
df_gesamt = pd.merge(df_gesamt, dwa, how = "outer", on = ["Digi_Index"])
df_gesamt["coord"] = df_gesamt["LONG"].astype(str) + ", " + df_gesamt["LAT"].astype(str)

In [5]:
for word in words:
    org = []
    item = []
    phontype = []
    lextype = []

    for i in range(len(df_gesamt)):
        leven(df_gesamt[f"maurer_org_{word}"][i], df_gesamt[f"dwa_org_{word}"][i], org)
        leven(df_gesamt[f"maurer_item_{word}"][i], df_gesamt[f"dwa_item_{word}"][i], item)
        leven(df_gesamt[f"maurer_phontype_{word}"][i], df_gesamt[f"dwa_phontype_{word}"][i], phontype)
        leven(df_gesamt[f"maurer_lextype_{word}"][i], df_gesamt[f"dwa_lextype_{word}"][i], lextype)

    df_gesamt[f"Levenshtein_org_{word}"] = org
    df_gesamt[f"Levenshtein_item_{word}"] = item
    df_gesamt[f"Levenshtein_phontype_{word}"] = phontype
    df_gesamt[f"Levenshtein_lextype_{word}"] = lextype

In [6]:
df_neu = pd.DataFrame()
df_neu["Fragebogen_Maurer"] = df_gesamt["Fragebogen-Nr._Maurer"]
df_neu["Fragebogen_DWA"] = df_gesamt["Digi_Index"]
df_neu["Ort"] = df_gesamt["Ort"]
df_neu["LONG"] = df_gesamt["LONG"]
df_neu["LAT"] = df_gesamt["LAT"]
df_neu["Koordinaten"] = df_gesamt["coord"]
df_neu["Lehrer_übereinstimmend"] = df_gesamt["Lehrer.übereinstimmend"]
df_neu["Lehrer_Einsender"] = df_gesamt["übereinst..Lehrer.ist.Einsender"]
df_neu["Lehrer_Beantworter"] = df_gesamt["übereinst..Lehrer.ist.Beantworter"]

for word in words:
    df_neu[f"maurer_org_{word}"] = df_gesamt[f"maurer_org_{word}"]
    df_neu[f"dwa_org_{word}"] = df_gesamt[f"dwa_org_{word}"]
    df_neu[f"Levenshtein_org_{word}"] = df_gesamt[f"Levenshtein_org_{word}"]
    df_neu[f"maurer_item_{word}"] = df_gesamt[f"maurer_item_{word}"]
    df_neu[f"dwa_item_{word}"] = df_gesamt[f"dwa_item_{word}"]
    df_neu[f"Levenshtein_item_{word}"] = df_gesamt[f"Levenshtein_item_{word}"]
    df_neu[f"maurer_phontype_{word}"] = df_gesamt[f"maurer_phontype_{word}"]
    df_neu[f"dwa_phontype_{word}"] = df_gesamt[f"dwa_phontype_{word}"]
    df_neu[f"Levenshtein_phontype_{word}"] = df_gesamt[f"Levenshtein_phontype_{word}"]
    df_neu[f"maurer_lextype_{word}"] = df_gesamt[f"maurer_lextype_{word}"]
    df_neu[f"dwa_lextype_{word}"] = df_gesamt[f"dwa_lextype_{word}"]    
    df_neu[f"Levenshtein_lextype_{word}"] = df_gesamt[f"Levenshtein_lextype_{word}"]

/tmp/ipykernel_14905/3088628376.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_neu[f"dwa_phontype_{word}"] = df_gesamt[f"dwa_phontype_{word}"]
/tmp/ipykernel_14905/3088628376.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_neu[f"Levenshtein_phontype_{word}"] = df_gesamt[f"Levenshtein_phontype_{word}"]
/tmp/ipykernel_14905/3088628376.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns

In [7]:
# Datenbereinigung
df_neu = df_neu.fillna("na")

# DataFrame nach Koordinaten zusammenfassen
df3 = df_neu.groupby('Koordinaten').min()

In [8]:
# Koordinaten in float datatype umwandeln
df3["LONG"] = [x.replace(",", ".") for x in df3["LONG"]]
df3["LAT"] = [x.replace(",", ".") for x in df3["LAT"]]
df3[['LONG', 'LAT']] = df3[['LONG', 'LAT']].astype(float)

In [9]:
levenshtein_item = []
levenshtein_phon= []
levenshtein_lex = []
for word in words:
    df = pd.read_csv(f"/home/kopatsch/Masterarbeit/MA/masterarbeit/data/levenshtein_single_files/{word}.csv", sep= "\t")
    df4 = df.drop(df[df["Levenshtein Item Min"] == 999].index)
    liste1 = list(df4["Levenshtein Item Min"])
    for x in liste1:
        levenshtein_item.append(x)
        levenshtein_phon.append(x)
        levenshtein_lex.append(x)
    df5 = df.drop(df[df["Levenshtein Phontype Min"] == 999].index)
    liste2 = list(df5["Levenshtein Phontype Min"])
    for x in liste2:
        levenshtein_phon.append(x)
        levenshtein_lex.append(x)
    df6 = df.drop(df[df["Levenshtein Lextype Min"] == 999].index)
    liste3 = list(df6["Levenshtein Lextype Min"])
    for x in liste3:
        levenshtein_lex.append(x)


mittel_item = np.mean(levenshtein_item)
stvar_item = np.std(levenshtein_item)
mittel_phon = np.mean(levenshtein_phon)
stvar_phon = np.std(levenshtein_phon)
mittel_lex = np.mean(levenshtein_lex)
stvar_lex = np.std(levenshtein_lex)


index = 14
for word in words:
    df3.insert(loc = index, column = f"z_score_item_{word}", value = ((df3[f"Levenshtein_item_{word}"]-mittel_item)/stvar_item))
    index += 4
    df3.insert(loc = index, column = f"z_score_phontype_{word}", value = ((df3[f"Levenshtein_phontype_{word}"]-mittel_phon)/stvar_phon))
    index += 4
    if index < 217:
        df3.insert(loc = index, column = f"z_score_lextype_{word}", value = ((df3[f"Levenshtein_lextype_{word}"]-mittel_lex)/stvar_lex))
        index += 7
    else:
        df3["z_score_lextype_zimmerfliege"] = ((df3[f"Levenshtein_lextype_{word}"]-mittel_lex)/stvar_lex)

# speichern
df3.to_csv("/home/kopatsch/Masterarbeit/MA/masterarbeit/data/teacher/teacher_with_words.csv", sep = "\t", index = False)

/tmp/ipykernel_14905/2538195872.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df3.insert(loc = index, column = f"z_score_item_{word}", value = ((df3[f"Levenshtein_item_{word}"]-mittel_item)/stvar_item))
/tmp/ipykernel_14905/2538195872.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df3.insert(loc = index, column = f"z_score_phontype_{word}", value = ((df3[f"Levenshtein_phontype_{word}"]-mittel_phon)/stvar_phon))
/tmp/ipykernel_14905/2538195872.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usuall